In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 设置随机种子以保证可重复性
torch.manual_seed(42)

# 读取 csv_soh.csv
df_soh = pd.read_csv('csv_soh.csv', skiprows=1)  # 跳过第一行（表头）

# 提取列
labels = torch.tensor(df_soh.iloc[:, 0].values, dtype=torch.float32).view(-1, 1)  # 使用第一列作为标签
inputs = torch.tensor(df_soh.iloc[:, 1:3].values, dtype=torch.float32)  # 使用第二和第三列作为输入特征

# 将数据集拆分为训练集和测试集（80% 训练，20% 测试）
inputs_train, inputs_test, labels_train, labels_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)

# 转换为 PyTorch 数据集
train_dataset = TensorDataset(inputs_train, labels_train)
test_dataset = TensorDataset(inputs_test, labels_test)

# 创建训练集和测试集的 DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义一个简单的神经网络模型
class SimpleDNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleDNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化模型
input_size = 2  # 输入特征的数量
hidden_size = 8  # 隐藏层单元的数量
output_size = 1  # 输出单元的数量（回归问题通常为1）

model = SimpleDNN(input_size, hidden_size, output_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for inputs_batch, labels_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 测试模型
model.eval()
with torch.no_grad():
    test_loss = 0.0
    predictions = []

    # 选择测试集的子集以打印预测
    num_samples_to_print = 10
    sample_indices = np.random.choice(len(test_loader.dataset), num_samples_to_print, replace=False)

    for i, (inputs_batch, labels_batch) in enumerate(test_loader):
        outputs = model(inputs_batch)
        test_loss += criterion(outputs, labels_batch).item()

        # 存储选择子集的预测
        if i in sample_indices:
            predictions.append((outputs.numpy(), labels_batch.numpy()))

    average_test_loss = test_loss / len(test_loader)
    print(f'Average Test Loss: {average_test_loss:.4f}')

# 打印选择子集的预测和实际值
for pred, actual in predictions:
    print(f'Prediction: {pred.flatten()}, Actual: {actual.flatten()}')


Epoch [1/3], Loss: 0.0001
Epoch [2/3], Loss: 0.0000
Epoch [3/3], Loss: 0.0000
Average Test Loss: 0.0000


In [38]:
# Print model parameters
for name, param in model.named_parameters():
    print(f'Parameter name: {name}, Size: {param.size()}, Values: {param}')


Parameter name: fc1.weight, Size: torch.Size([8, 2]), Values: Parameter containing:
tensor([[ 0.4426,  0.4714],
        [-0.2552,  0.5521],
        [-0.2313,  0.0666],
        [-0.4389,  0.3065],
        [ 0.6233, -0.5188],
        [ 0.5141,  0.0091],
        [ 0.6376,  0.2475],
        [ 0.3005, -0.1364]], requires_grad=True)
Parameter name: fc1.bias, Size: torch.Size([8]), Values: Parameter containing:
tensor([ 0.4351,  0.0093, -0.4065,  0.0758, -0.3258, -0.1990, -0.1471,  0.4288],
       requires_grad=True)
Parameter name: fc2.weight, Size: torch.Size([1, 8]), Values: Parameter containing:
tensor([[-0.1524, -0.0344,  0.0045, -0.0887,  0.0334, -0.2610,  0.4916, -0.2634]],
       requires_grad=True)
Parameter name: fc2.bias, Size: torch.Size([1]), Values: Parameter containing:
tensor([0.3993], requires_grad=True)


In [11]:
import os
import pandas as pd

# ... (previous code remains unchanged)

# Create a directory to store the parameters CSV files
output_dir = 'model_parameters'
os.makedirs(output_dir, exist_ok=True)

# Save model parameters to CSV files
for idx, (name, param) in enumerate(model.named_parameters()):
    layer_name = name.split('.')[0]
    layer_params = param.detach().numpy().flatten()

    # Create a DataFrame and save to CSV
    df = pd.DataFrame({f'{layer_name}_param_{i}': layer_params[i] for i in range(len(layer_params))}, index=[0])
    csv_path = os.path.join(output_dir, f'{layer_name}_params_{idx}.csv')
    df.to_csv(csv_path, index=False)

    print(f'Parameters for {layer_name} saved to {csv_path}')

print('Parameters exported successfully.')


Parameters for fc1 saved to model_parameters\fc1_params_0.csv
Parameters for fc1 saved to model_parameters\fc1_params_1.csv
Parameters for fc2 saved to model_parameters\fc2_params_2.csv
Parameters for fc2 saved to model_parameters\fc2_params_3.csv
Parameters exported successfully.


In [79]:
def relu(x):
    return np.max(x, 0)
w1 = np.array([[ 0.4426,  0.4714],
        [-0.2552,  0.5521],
        [-0.2313,  0.0666],
        [-0.4389,  0.3065],
        [ 0.6233, -0.5188],
        [ 0.5141,  0.0091],
        [ 0.6376,  0.2475],
        [ 0.3005, -0.1364]])
w2 = np.array([-0.1524, -0.0344,  0.0045, -0.0887,  0.0334, -0.2610,  0.4916, -0.2634])

b1 = np.array([ 0.4351,  0.0093, -0.4065,  0.0758, -0.3258, -0.1990, -0.1471,  0.4288])
b2 = np.array([0.3993])

label = np.array([0.77])
data_in = np.array([3.74739301071869,32.0753931146082])


def relu(x):
    return np.maximum(x, 0)

# 输入

# 第一层计算
layer1_output = relu(np.dot(data_in, w1.T) + b1)

# 第二层计算
output = np.dot(layer1_output, w2) + b2
print(np.dot(data_in, w1.T) + b1)
# 输出
print("输出:", output)


[ 17.21403646  16.76178984   0.86294918   8.2621772  -14.63076388
   2.01942082  10.18089758  -2.82019202]
输出: [0.94816384]


In [81]:
a_int = np.round(data_in * 2**16).astype(int)
print(a_int)
layer1_output = relu(np.dot(a_int, w1_int.T)/2**16 + b1_int)
print(layer1_output/2**16)
# 第二层计算
output = np.dot(layer1_output, w2_int)/2**16 + b2_int
print(output/2**16)

[ 245589 2102093]
[0.21418805 0.         0.         0.26227547 0.         0.01922644
 0.18083733 0.17988845]
[0.37989832]


In [75]:
import numpy as np

k=16
m=32
# 定义一个函数将浮点数转换为32位二进制字符串
def float_to_bin(num):
    # 处理负数的补码表示
    if num < 0:
        num =  num + 2**m
    # 转换为32位二进制字符串
    bin_str = bin(num)[2:].zfill(m)
    return bin_str

# 给定的参数
w1 = np.array([[ 0.4426,  0.4714],
                [-0.2552,  0.5521],
                [-0.2313,  0.0666],
                [-0.4389,  0.3065],
                [ 0.6233, -0.5188],
                [ 0.5141,  0.0091],
                [ 0.6376,  0.2475],
                [ 0.3005, -0.1364]])

w2 = np.array([-0.1524, -0.0344,  0.0045, -0.0887,  0.0334, -0.2610,  0.4916, -0.2634])

b1 = np.array([ 0.4351,  0.0093, -0.4065,  0.0758, -0.3258, -0.1990, -0.1471,  0.4288])
b2 = np.array([0.3993])

# 将浮点数参数转换为32位整数
w1_int = np.round(w1 * (2**k)).astype(int)
w2_int = np.round(w2 * (2**k)).astype(int)
b1_int = np.round(b1 * (2**k)).astype(int)
b2_int = np.round(b2 * (2**k)).astype(int)

# 打印模型参数的二进制表示
print("weights_fc1:")
for weights_row in w1_int:
    for weight in weights_row:
        print(f"{float_to_bin(weight)}\t{weight}")

print("\nbias_fc1:")
for bias in b1_int:
    print(f"{float_to_bin(bias)}\t{bias}")

print("\nweights_fc2:")
for weight in w2_int:
    print(f"{float_to_bin(weight)}\t{weight}")

print("\nbias_fc2:")
for bias in b2_int:
    print(f"{float_to_bin(bias)}\t{bias}")


weights_fc1:
00000000000000000111000101001110	29006
00000000000000000111100010101110	30894
11111111111111111011111010101011	-16725
00000000000000001000110101010110	36182
11111111111111111100010011001010	-15158
00000000000000000001000100001101	4365
11111111111111111000111110100100	-28764
00000000000000000100111001110111	20087
00000000000000001001111110010001	40849
11111111111111110111101100110000	-34000
00000000000000001000001110011100	33692
00000000000000000000001001010100	596
00000000000000001010001100111010	41786
00000000000000000011111101011100	16220
00000000000000000100110011101110	19694
11111111111111111101110100010101	-8939

bias_fc1:
00000000000000000110111101100011	28515
00000000000000000000001001100001	609
11111111111111111001011111110000	-26640
00000000000000000001001101101000	4968
11111111111111111010110010011000	-21352
11111111111111111100110100001110	-13042
11111111111111111101101001011000	-9640
00000000000000000110110111000110	28102

weights_fc2:
1111111111111111110110001